In [1]:
### QUERY CACHING WITH REDIS

In [145]:
# Import the necessary libraries
import mysql.connector
import redis
import hashlib
import pickle
import random

## Query Caching Function

In [146]:
# Define a function to execute a SQL query and cache it in Redis

def execute_query(query):
  # First, check if the query has been previously executed and the result is in the cache
    query_hash = hashlib.sha1(query.encode()).hexdigest()
    if query_hash in cache:
        print("Retrieving result from cache")
        # Deserialize the result from the cache
        result = pickle.loads(r.get(query_hash))
        #print(result)
        return result
    
    # If the query has not been previously executed, execute it now
    #print("Executing query")
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Serialize the result before storing it in the cache using Redis
    #result_str = pickle.dumps(result)
    #print(result_str)
    #r.set(query_hash, result_str)
    
    
    # Create a pipeline object
    pipeline = r.pipeline()
    
    # Use the pipeline to set the query result in the cache
    pipeline.set(query_hash, pickle.dumps(result))
    # Set a time to live for the string in the cache
    pipeline.expire(query_hash, 600)
    
    # Execute the pipeline
    pipeline.execute()
    
    
    
    # Set a time to live for the string in the cache
    #r.expire(query_hash, 60)
    
    # Return the result of the query
    return result

## Connect to MySQL Db and Redis

In [147]:
# Create a connection to the database
# connect to the MySQL server
conn = mysql.connector.connect(
                                user='admin', 
                                password='password',
                                host='44.229.176.207',
                                database='classicmodels'
                                )


# Create a connection to the Redis server
r = redis.Redis(host="redis-11919.bamos-west.demo.redislabs.com", port=11919)

### Query Generator

In [148]:
# define the query with placeholders for the dynamic values
query = """
    SELECT 
        c.customerNumber,
        c.customerName,
        c.contactLastName,
        c.contactFirstName,
        c.phone,
        c.addressLine1,
        c.addressLine2,
        c.city,
        c.state,
        c.postalCode,
        c.country,
        c.salesRepEmployeeNumber,
        c.creditLimit,
        o.orderNumber,
        o.orderDate,
        o.requiredDate,
        o.shippedDate,
        o.status,
        o.comments,
        o.customerNumber
    FROM
    customers c
LEFT JOIN orders o 
    ON c.customerNumber = o.customerNumber
    WHERE
    o.orderNumber > {} AND o.orderNumber < {}
    LIMIT {}
    ;
"""

# create an empty list for the formatted queries
formatted_queries = []

# loop n times
for i in range(1000):
    # generate a random integer for the creditlimit value
    minOrderNumber = random.randint(10099, 10262)
    maxOrderNumber = random.randint(10263, 10426)
    limit          = random.randint(0, 326)

    # format the query with the dynamic values
    random_query = query.format(minOrderNumber, maxOrderNumber, limit)
    
    # define the list of fields to add or drop from the query
    fields = [
        "c.customerName,",
        "c.contactLastName,",
        "c.contactFirstName,",
        "c.phone,",
        "c.addressLine1,",
        "c.addressLine2,",
        "c.city,",
        "c.state,",
        "c.postalCode,",
        "c.country,",
        "c.salesRepEmployeeNumber,",
        "c.creditLimit,",
        "o.orderDate,",
        "o.requiredDate,",
        "o.shippedDate,",
        "o.status,",
        "o.comments,"
    ]
    
    # generate a random number to determine whether to add or drop fields
    rand = random.randint(0, 1)

    # if the number is 0, drop a random field from the query
    if rand == 0:
        # choose a random field to drop
        chosen_values = random.sample(fields,3)
        # remove the field from the list of available fields
        # Remove the chosen values from the list
        for value in chosen_values:
            fields.remove(value)
            
            # Remove values from query
            for field in fields:
                random_query = random_query.replace(field, "")
        # remove white space        
        formatted_query = " ".join(random_query.split())
    else:
        # else, use the random query
        # remove white space 
        formatted_query = " ".join(random_query.split())
    
    # append the formatted query to the list
    formatted_queries.append(formatted_query)

print(formatted_queries[0:5])
print(len(formatted_queries))

['SELECT c.customerNumber, c.customerName, c.contactLastName, c.contactFirstName, c.phone, c.addressLine1, c.addressLine2, c.city, c.state, c.postalCode, c.country, c.salesRepEmployeeNumber, c.creditLimit, o.orderNumber, o.orderDate, o.requiredDate, o.shippedDate, o.status, o.comments, o.customerNumber FROM customers c LEFT JOIN orders o ON c.customerNumber = o.customerNumber WHERE o.orderNumber > 10177 AND o.orderNumber < 10285 LIMIT 168 ;', 'SELECT c.customerNumber, c.addressLine2, o.orderNumber, o.customerNumber FROM customers c LEFT JOIN orders o ON c.customerNumber = o.customerNumber WHERE o.orderNumber > 10234 AND o.orderNumber < 10274 LIMIT 158 ;', 'SELECT c.customerNumber, c.postalCode, o.orderNumber, o.customerNumber FROM customers c LEFT JOIN orders o ON c.customerNumber = o.customerNumber WHERE o.orderNumber > 10226 AND o.orderNumber < 10302 LIMIT 190 ;', 'SELECT c.customerNumber, c.customerName, o.orderNumber, o.customerNumber FROM customers c LEFT JOIN orders o ON c.custom

## Run Queries Through Cache Function

In [149]:
# Create a cache to store the results of previous queries
cache = {}

# loop n times
for i in range(1000):

    chosen_query = random.choice(formatted_queries)

    result = execute_query(chosen_query)
    #print(result)

print('new')
# loop n times
for i in range(1000):

    chosen_query = random.choice(formatted_queries)

    result = execute_query(chosen_query)
    #print(result)

new
